In [1]:
x=1

In [2]:
import scanpy as sc
import pandas as pd

In [3]:
patient = 'donor_id'
condition = 'lung_condition_coarse'
n_splits = 5

In [4]:
adata = sc.read('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/snakemake/hlca/5fold/hlca_hyperparam/data/pp/hlca_hvg_pearson.h5ad')
adata

AnnData object with n_obs × n_vars = 2149684 × 2000
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_l

In [5]:
adata.X.data

array([1., 5., 1., ..., 1., 2., 3.], dtype=float32)

# Random forest

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
adata.obs[condition] = adata.obs[condition].astype('category')
rename_dict = {name: number for number, name in enumerate(sorted(list(adata.obs[condition].cat.categories)))}
rename_dict

{'CF': 0,
 'COPD': 1,
 'COVID-19': 2,
 'Carcinoma': 3,
 'Chronic rhinitis': 4,
 'Healthy': 5,
 'ILD': 6,
 'IPF': 7,
 'Pneumonia non-COVID': 8}

In [9]:
val_accuracies = []
val_avg = []
for i in range(n_splits):
    print(f'Processing split = {i}...')
    #df = adata.obs[[f'split{i}', patient]].drop_duplicates()
    #train = list(df[df[f'split{i}'] == 'train'][patient])
    #val = list(df[df[f'split{i}'] == 'val'][patient])
    # train data
    x = pd.DataFrame(adata[adata.obs[f'split{i}'] == 'train'].X.A).to_numpy()
    num_of_classes = len(adata.obs[condition].cat.categories)
    y = adata[adata.obs[f'split{i}'] == 'train'].obs[condition].cat.rename_categories(rename_dict)
    y = y.to_numpy()
    print("Train shapes:")
    print(f"x.shape = {x.shape}")
    print(f"y.shape = {y.shape}")
    # val data
    x_val = pd.DataFrame(adata[adata.obs[f'split{i}'] == 'val'].X.A).to_numpy()
    y_val = adata[adata.obs[f'split{i}'] == 'val'].obs[condition].cat.rename_categories(rename_dict)
    y_val = y_val.to_numpy()
    print("Val shapes:")
    print(f"x_val.shape = {x_val.shape}")
    print(f"y_val.shape = {y_val.shape}")
    # fit
    X = x
    Y = y
    clf = RandomForestClassifier(n_estimators=5)
    clf.fit(X, Y)
    print(f'Train accuracy = {np.sum(clf.predict(X) == Y)/len(Y)}.')
    y_pred = clf.predict(x_val)
    val_accuracy = np.sum(y_pred == y_val)/len(y_val)
    
    df = classification_report(y_val, y_pred, output_dict=True)
    df = pd.DataFrame(df).T
    print('class report')
    print(df)
    
    val_accuracies.append(df["f1-score"]["accuracy"])
    val_avg.append(df["f1-score"]["weighted avg"])
    
    val_accuracy = df["f1-score"]["accuracy"]
    
    print(f'Val accuracy = {val_accuracy}.')
    print('===========================')


print(f"Mean validation accuracy across 5 CV splits for a random forest model = {np.mean(np.array(val_accuracies))}.")
print(f"Mean validation weighted avg across 5 CV splits for a random forest model = {np.mean(np.array(val_avg))}.")


Processing split = 0...
Train shapes:
x.shape = (1733726, 2000)
y.shape = (1733726,)
Val shapes:
x_val.shape = (415958, 2000)
y_val.shape = (415958,)
Train accuracy = 0.9861719787325102.
class report
              precision    recall  f1-score        support
0              0.286445  0.228106  0.253968     982.000000
1              0.581154  0.371824  0.453499   24681.000000
2              0.740586  0.826728  0.781290   36134.000000
3              0.558565  0.695205  0.619439   11867.000000
4              0.929354  0.800510  0.860134    6672.000000
5              0.783435  0.921518  0.846885  243457.000000
6              0.210013  0.258778  0.231859    8316.000000
7              0.719248  0.371795  0.490197   78979.000000
8              0.553621  0.124025  0.202651    4870.000000
accuracy       0.743669  0.743669  0.743669       0.743669
macro avg      0.595824  0.510943  0.526658  415958.000000
weighted avg   0.736120  0.743669  0.722605  415958.000000
Val accuracy = 0.7436688319493795